In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas
import os

Mounted at /content/drive


In [6]:
files = os.listdir()

In [7]:
%cd /content/drive/MyDrive/466\ Project/CSC466-Project

/content/drive/MyDrive/466 Project/CSC466-Project


In [8]:
!pwd

/content/drive/MyDrive/466 Project/CSC466-Project


In [9]:
def get_hash_info(df):
  hash_list = []
  hash_type = []
  len_list = []
  for i in range(len(df)):
    rand_col = (np.random.choice(["SHA-256", "SHA-1", "MD5","SHA-3-256"]))
    hash_list.append(df.iloc[i][rand_col])
    hash_type.append(rand_col)
    len_list.append(len(df.iloc[i][rand_col]))
  return pd.Series(hash_list), pd.Series(hash_type), pd.Series(len_list)


In [11]:
# processed unprocessed files
import pandas as pd
import numpy as np

import os
files = ["pdfHash.csv", "jpgHash.csv"]
df = pd.DataFrame()
for file in files:
  df = df.append(pd.read_csv(file))

df["hash"], df["hash_type"], df["hash_len"] = get_hash_info(df)
df = df.drop(columns=["SHA-256", "SHA-1", "MD5", "SHA-3-256"])

display(df.head())



,filename,file size,file type,file dist of bytes,consecutiveByte,consecutiveCount,hash,hash_type,hash_len
0,/mnt/f/pdfs/100G Networking Technology Overvie...,1.516903,.pdf,"[16562, 6642, 6021, 6415, 5546, 5398, 5934, 58...",b'\x10',50,3802d97d4aa5d9b65936337cbb06984e4df66bb6,SHA-1,40
1,/mnt/f/pdfs/2018 CppCon Unwinding the Stack - ...,1.164143,.pdf,"[12322, 4160, 4338, 3771, 4315, 4141, 3978, 39...",b'\x00',733,183cab0074049265b7d52a0e54e7a7e4,MD5,32
2,/mnt/f/pdfs/37 Million Compilations - Investig...,0.459589,.pdf,"[1387, 1746, 1593, 1665, 1486, 1558, 1698, 163...",b'0',10,e86d07e561a5405af36323cd567731d6365635d9af4011...,SHA-3-256,64
3,/mnt/f/pdfs/5-Level Paging and 5-Level EPT - I...,0.199550,.pdf,"[733, 665, 634, 628, 625, 687, 649, 662, 637, ...",b' ',100,18de0224a4567a9ae17ff756e29bb1318c9adbd4,SHA-1,40
4,/mnt/f/pdfs/5-Level Paging and 5-Level EPT - I...,0.187398,.pdf,"[736, 658, 666, 670, 627, 673, 645, 660, 669, ...",b' ',100,adb2a5e99912c005e416f5099add45e39772d360685394...,SHA-256,64


In [ ]:
a = [1,2,3,4]
b = [2,5,6]

for i in b and not in a:
  print(i)

SyntaxError: ignored

# TO DO

* Count total number of chars for each file
* Pick random hash out of available hashes; record both hash and name of hash
* Split hash into groups/chunks (3, maybe 5)
* Count length of hash
* Count number of 0s/1s in hash
* Distribution of 0-e in hash